# Modello NRP
## Fabio Ciccarelli, matricola n°1835348

### Step 0

Importing needed libraries.

In [1]:
import pyomo.environ as pe
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Alignment
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import math

try:
    import ipywidgets as widgets
except:
    !pip install ipywidgets

from ipywidgets import interact


### Step 0.1

Defining a function which returns KPIs for a given solution.

In [2]:
'''The following function takes a Pyomo model as parameter and returns some Key Performance Indicators (KPIs) about it,
    such as information about the workload or about overtime hours. Moreover, if problem is not admissible, it returns
    an alert message''';

def KPI(model):
    
    if sum(model.j[s,t] for s in S for t in T)!=0:
        print('This problem has no admissible solutions. Please try to change parameters or hire extra nurses.')
        return
    
    else:
        workload_list = []
        hours_per_week = {}

        for k in K:
            dict_h = {}
            for i in I:
                dict_h[i] = 8*sum(model.x[i,s,t]() for s in S for t in weeks[k])

            hours_per_week[k] = dict_h

        max_weekly = 0

        for k in K:
            if max(hours_per_week[k].values())>max_weekly:
                max_weekly = max(hours_per_week[k].values())

        min_weekly = 1e6

        for k in K:
            if min(hours_per_week[k].values())<min_weekly:
                min_weekly = min(hours_per_week[k].values())


        for i in I:
            workload_list.append(sum(model.x[i,s,t]()*W.at[t,s] for s in S for t in T))

        print('KPI: max work-load ----> ', max(workload_list))
        print('*'*100)
        print('KPI: min work-load ----> ', min(workload_list))
        print('*'*100)
        print('KPI: max hours worked weekly ----> ', max_weekly)
        print('*'*100)
        print('KPI: min hours worked weekly ----> ', min_weekly)
        print('*'*100)
        print('KPI: overtime hours ---->', sum(model.ot[i,k]() for i in I for k in K))
        print('*'*100)



        return 

In [3]:
'''This function works the same way as KPI function does, but it also returns a dictionary (or a list, according to the
    boolean value "dictionary") with a summary of those Performance Indicators. It is crucial when it comes to make plots.''';

def KPI_dict(model, dictionary = False):
    workload_list = []
    hours_per_week = {}
    
    for k in K:
        dict_h = {}
        for i in I:
            dict_h[i] = 8*sum(model.x[i,s,t]() for s in S for t in weeks[k])
        
        hours_per_week[k] = dict_h
        
    max_weekly = 0
    
    for k in K:
        if max(hours_per_week[k].values())>max_weekly:
            max_weekly = max(hours_per_week[k].values())
            
    min_weekly = 1e6
    
    for k in K:
        if min(hours_per_week[k].values())<min_weekly:
            min_weekly = min(hours_per_week[k].values())
            
            
    for i in I:
        workload_list.append(sum(model.x[i,s,t]()*W.at[t,s] for s in S for t in T))
     
    if dictionary == False:
        return [max(workload_list), min(workload_list), max_weekly, min_weekly, 
               sum(model.ot[i,k]() for i in I for k in K)]
    if dictionary == True:
        return {'MWL':max(workload_list), 
               'mWL':min(workload_list),
               'MHW': max_weekly,
               'mHW':min_weekly,
               'OT':sum(model.ot[i,k]() for i in I for k in K)}

### Step 1.0

Importing useful data from 'Data.xlsx' and initializing some sets and data frames.

In [4]:
Personnel = pd.read_excel('Data.xlsx', index_col = 0, sheet_name = 'Personnel')  # Importing the personnel sheet
                                                                
I = Personnel.index  # Initializing nurses' set

Patients = pd.read_excel('Data.xlsx', index_col = 0, sheet_name = 'Patients') # Importing the sheet relative to daily demand
        
T = Patients.index   # Initializing the time-frame set 

S = ['Morning', 'Afternoon', 'Night']  # Initializing the daily shifts
        
R = pd.DataFrame(index = T, columns = S) 




W = R.copy()    # Creating a dataframe to set shifts' weight
    
for t in T:
    for s in S:
        if 'Saturday' in t or 'Sunday' in t:     
            W.at[t, s] = 2
        else:
            W.at[t,s] = 1             # Weekends weigh twice as much than weekdays   
                
for t in T:
    for s in S:
        if s=='Night':
            W.at[t,s] = W.at[t,s] * 2     # Nights weigh twice as much than diurnal shifts
            


'''
Consider that night shifts on weekend days weigh four times as much than diurnal shifts on weekdays.
''';

### Step 1.1

Defining some parameters and $\textbf{day_after}$ dictionary. 

In [5]:
h_min = 24
h_max = 40  # Creating parameters 'minimum weekly working hours' and 'maximum weekly working hours'

a = {'Morning' : 5,
    'Afternoon' : 5,
    'Night' : 10}    # How many patients can a single nurse take care of during a given shift?
                  # Inserted values are purely arbitrary.


days = list(T)
weeks={}  
    
    
for j in range(0, len(days), 7):
    weeks[int(j/7+1)] = []
    for k in range(j, j+7):
        weeks[int(j/7+1)].append(days[k])


'''
We create a week list because days that make up the T period are not supposed to be represented as
an ordered sequence of natural numbers, but as strings like 'Monday, 1 March', 'Tuesday, 2 March', ...
We therefore need to divide the period of time taken into account into weeks. Note that this procedure fails if in the file 
'Data.xlsx' we define a time frame T whose cardinality is not multiple of 7. 
It seems to us credible, however, that staff sheduling is defined on a weekly basis.

''';

K = weeks.keys()

day_after = {}

for j in range(0, len(days)-1):
    day_after[days[j]] = (days[j+1])   # We could define a function as well, instead of a dictionary.
                                       # This dictionary is crucial for constraint rest_after_night (see below)    

### Step 2.0

Mathematical formulation of the problem.


\
$$
\begin{align}
{min} \quad z \ \  + &\ \ p_1\sum_{i \in I} \sum_{k=1}^{q} {ot_{i,k}} \ + \ M \sum_{s \in S} \sum_{t \in T} {j_{s,t}} \\
\hspace{3 mm}\\
s.t.\hspace{13 mm} &\sum_{s \in S} x_{i,s,t} \leq 1 \quad & {i \in I,\ t \in T} \hspace{7 mm} & \tiny\text{(At most 1 shift a day)} \\
8&\sum_{s \in S} \sum_{t \in T_k} {x_{i,s,t}} \geq h_{min} \quad & {i \in I, \ k =1,...,q} \hspace{7 mm} & \tiny\text{(At least $h_{min}$ hours a week)}\\
a_s ( & \sum_{i \in I} {x_{i,s,t}} \ + \ j_{s,t}) \geq d_t \quad & {s \in S, \ t \in T} \hspace{7 mm} & \tiny\text{(Demand covering)}\\
& \sum_{s \in S} {x_{i,s,t+1}} \leq 1 - x_{i,s3,t} \quad & {i \in I, \ t \in T} \hspace{7 mm} & \tiny\text{(Rest after night shifts)}\\ 
8&\sum_{s \in S} \sum_{ t \in T_k} {x_{i,s,t}} - h_{max} \leq ot_{i,k} \quad &{i \in I, \ k=1,...,q} \hspace{7 mm} &\tiny\text{(Overtime hours)}\\
&\sum_{s \in S}\sum_{t \in T}{x_{i,s,t} \ w_{s,t}} \leq z \quad & {i \in I} \hspace{7 mm} & \tiny\text {(Maximum workload)}\\
\hspace{4 mm}\\
\hspace{4 mm}\\
& x_{i,s,t} \in \{0,1\} \hspace{4 mm}& {i \in I, \ s \in S,\ t \in T}\\
& ot_{i,k} \in \mathbb{N} &{ i \in I, \ k = 1,...,q}\\
& j_{s,t} \in \mathbb{N} & {s \in S, t \in T}\\
& z \in \mathbb{R_{+}}\\
\end{align}    
$$
​


In [6]:
'''
create_model function creates a model with the mathematical structure illustrated above. It is useful when we have to
solve iteratively this problem, with different input data. The parameter p (the cost of a single overtime hour) is set to 100 
by default.
''';

def create_model(p = 100):
    
    global m
    
    m = pe.ConcreteModel('NRP')

    # VARIABLES

    m.x = pe.Var(I, S, T, within = pe.Binary) 

    m.ot = pe.Var(I, K, within = pe.NonNegativeIntegers)   
    
    m.z = pe.Var(within = pe.NonNegativeReals)    
    
    m.j = pe.Var(S, T, within = pe.NonNegativeIntegers)    




    # OBJECTIVE FUNCTION    

    m.obj_fun = pe.Objective (expr = m.z + p*sum(m.ot[i, k] for i in I for k in K) + 1e6*sum(m.j[s,t] for s in S for t in T))

    





    # CONSTRAINTS    

    def max_daily(m, i, t):
        return sum(m.x[i, s, t] for s in S) <= 1

    m.max_daily = pe.Constraint (I, T, rule = max_daily)            



    def min_weekly(m, i, k):
        return 8*sum(m.x[i, s, t] for s in S for t in weeks[k]) >= h_min

    m.min_weekly = pe.Constraint (I,K, rule = min_weekly)           



    def cover_demand(m, s, t):
        return a[s]*(sum(m.x[i,s,t] for i in I)+m.j[s,t]) >= Patients.at[t, 'd']

    m.cover_demand = pe.Constraint(S, T, rule = cover_demand)        



    def rest_after_night(m, i, t):
        if t == days[len(days)-1]:
            return pe.Constraint.Skip
        else:
            return sum(m.x[i, s, day_after[t]] for s in S) <= (1 - m.x[i, 'Night', t])

    m.rest_after_night = pe.Constraint(I, T, rule = rest_after_night)     



    def overtime_hours(m, i, k):
        return 8*sum(m.x[i, s, t] for s in S for t in weeks[k]) - h_max <= m.ot[i, k]

    m.overtime_hours = pe.Constraint(I, K, rule = overtime_hours)         
                                                                          


    def load_upper_bound(m, i):
        return sum(m.x[i, s, t]*W.at[t,s] for s in S for t in T) <= m.z

    m.load_upper_bound = pe.Constraint(I, rule = load_upper_bound)        
    
    
 

### Step 2.1

Solving the problem and presenting results.

In [7]:
create_model()
solver = pe.SolverFactory('cplex_direct')
solver.solve(m)

standard_results = m.obj_fun()

KPI(m)

'''
The code below is an idea of how output solutions could be represented and communicated to personnel. 
The idea is to obtain an Excel workbook (named Results.xlsx), with as many sheets as the nurses. In the i-th sheet there 
will be the shifts covered by the i-th nurse during T period. 

''';

'''
print()
for k in K:
    print('Total number of overtime hours in week {} is equal to {}'.format(k, sum(m.ot[i,k]() for i in I)))
    print('*'*100)
    print()



    
for i in I:
    for t in T:
        for s in S:
            R.at[t,s] = ' '

            
wb = openpyxl.Workbook() 
wb.save('Results.xlsx')
    
for i in I:
    for t in T:
        for s in S:
            if m.x[i,s,t]()==1:
                R.at[t,s] = 'x'
    with pd.ExcelWriter('Results.xlsx', mode='a', engine = 'openpyxl') as writer:
        R.to_excel(writer,  sheet_name = i)
    for t in T:
        for s in S:
            R.at[t,s] = ' '





wb = load_workbook('Results.xlsx')


for i in I:
    ws = wb[i]
    ws.column_dimensions['A'].width = 25
    for col in ws.columns:
        for cell in col:
            cell.alignment = Alignment (horizontal = 'center')
            

wb.remove(wb['Sheet']) # Eliminazione del foglio bianco inserito di default da openpyxl

wb.save('Results.xlsx')

''';


KPI: max work-load ---->  33.0
****************************************************************************************************
KPI: min work-load ---->  32.0
****************************************************************************************************
KPI: max hours worked weekly ---->  56.0
****************************************************************************************************
KPI: min hours worked weekly ---->  32.0
****************************************************************************************************
KPI: overtime hours ----> 72.0
****************************************************************************************************

Total number of overtime hours in week 1 is equal to 0.0
****************************************************************************************************

Total number of overtime hours in week 2 is equal to 48.0
****************************************************************************************************



In [8]:
'''
The following lines of code allow to show results in an interactive way, with the possibility for the user to choose 
a single nurse, a single day or a single shift.
''';

solution = pd.DataFrame(index = None, columns = ['Day','Shift','Nurse'])
for t in T:
     for s in S:
        for i in I:
            dp = {}
            if m.x[i,s,t]()==1:
                dp['Day']= t
                dp['Shift']= s
                dp['Nurse']= i
                solution = solution.append(dp, ignore_index=True)
                
                
def interactive(nurse, day, shift):
    
    if nurse == 'All':
        df_int = solution[solution['Nurse'].isin(I)]
    else:
        df_int = solution[solution['Nurse']==nurse]
    
    if day == 'All':
        df_int = df_int[df_int['Day'].isin(T)]
    else:
        df_int = df_int[df_int['Day']==day]
    
    if shift == 'All':
        df_int = df_int[df_int['Shift'].isin(S)]
    else:
        df_int = df_int[df_int['Shift']==shift]
    
    
    df_int = df_int.set_index(pd.Index(list(range(1, len(df_int.index)+1))))
    display(df_int)
    

def display_scheduling(display):
    if display == True:
        interact(interactive, nurse = widgets.Dropdown(value = 'All', options = list(I) + ['All']), 
                 day = widgets.Dropdown(value = 'All', options = list(T)+ ['All']), 
                 shift = widgets.Dropdown(value = 'All', options = list(S)+ ['All'])
                );
        
        
interact(display_scheduling, display = widgets.ToggleButton(value=False, description = 'Display'))



interactive(children=(ToggleButton(value=False, description='Display'), Output()), _dom_classes=('widget-inter…

<function __main__.display_scheduling(display)>

### Basic model statistics

In [9]:
avg_hours_per_week = {}

for i in I:
    avg_hours_per_week[i] = 8*sum(m.x[i,s,t]() for s in S for t in T)/len(K)
    
mvals = KPI_dict(m, dictionary = True).values()

$$
\begin{align}
\hspace{3 mm}\\
\hspace{3 mm}\\
\end{align}
$$

## Step 3
### Extra-nurses

The problem is solved by adding a progressive number of extra nurses and interactively establishing the cost of hiring each of them.


In [13]:
def set_weight(solve: bool, p1: int, c: int):
    
    global I
    
    
    if solve == 'Solve':
        
        create_model(p1)
        solver.solve(m)

        limit = sum(m.ot[i,k]() for i in I for k in K) 
        extra = 0
        solutions = {0: KPI_dict(m)}
        obj_values = {0: m.obj_fun()}
        while extra < 1e6:
            extra += 1
            I = I.insert(len(I),'Extra'+str(extra))
            create_model(p1)
            solver.solve(m)

            
            solutions[extra] = KPI_dict(m);
            obj_values [extra] = m.obj_fun() + c*extra
            
            if obj_values[extra] >= obj_values[extra-1]:
                break

        print()
        best_sol = None
        for sol in obj_values.keys():
            if obj_values[sol] == min(obj_values.values()):
                best_sol = sol
                if best_sol == 1:
                    print('THE BEST SOLUTION IS TO HIRE **{}** EXTRA-NURSE'.format(best_sol))
                else:
                    print('THE BEST SOLUTION IS TO HIRE **{}** EXTRA-NURSES'.format(best_sol))
                print('*'*100)
                print('These are the KPIs for the optimal solution: \n \
                KPI: max work-load ----> {} \n \
                ************************************************************************************** \n \
                KPI: min work-load ----> {} \n \
                ************************************************************************************** \n \
                KPI: max hours worked weekly ----> {} \n \
                ************************************************************************************** \n \
                KPI: min hours worked weekly ----> {} \n \
                ************************************************************************************** \n \
                KPI: overtime hours ----> {} \n \
                ************************************************************************************** \n \
                '.format(solutions[best_sol][0],
                         solutions[best_sol][1],
                         solutions[best_sol][2],
                         solutions[best_sol][3],
                         solutions[best_sol][4],))


        #print(obj_values)
        for i in range(best_sol+1, extra+1):
            I = I.drop('Extra'+str(i))
            
        create_model(p1)
        solver.solve(m)


        global solution_bis 
        global I_2
        I_2 = I.copy()

        solution_bis = pd.DataFrame(index = None, columns = ['Day','Shift','Nurse'])
        for t in T:
             for s in S:
                for i in I_2:
                    dp = {}
                    if m.x[i,s,t]()==1:     
                        dp['Day']= t
                        dp['Shift']= s
                        dp['Nurse']= i
                        solution_bis = solution_bis.append(dp, ignore_index=True)


        avg_hours_per_week_1 = {}

        for i in I:
            avg_hours_per_week_1[i] = 8*sum(m.x[i,s,t]() for s in S for t in T)/len(K) 

        plt.hist(avg_hours_per_week_1.values(), bins = 20)
        plt.title('Average working hours per week')
        plt.show()

        labels = KPI_dict(m, dictionary = True).keys()    
        mvalues = mvals
        m1values = KPI_dict(m, dictionary = True).values() 

        x = np.arange(len(labels))
        width = 0.35  
        fig, ax = plt.subplots()
        rects1 = ax.bar(x - width/2, mvalues, width, label='without extra-nurses')
        rects2 = ax.bar(x + width/2, m1values, width, label='with extra-nurses')

        ax.set_title('Differences in KPIs between the two optimal solutions')
        ax.set_xticks(x)
        ax.set_xticklabels(labels)
        ax.legend()

        plt.show()

        for i in I:
            if 'Extra' in i:
                I = I.drop(i)
            
    

    

    
interact(set_weight, 
         solve = widgets.ToggleButtons(options = ['Solve', 'Change Parameters'], description = ' '),
         p1 = widgets.IntSlider(value = 35, min = 0, max = 200, step = 1, orientation = 'horizontal', description = 'p1:'),
         c = widgets.IntSlider(value = 1000, min = 0, max = 5000, step = 1, orientation = 'horizontal', description = 'c:'),
        )

interactive(children=(ToggleButtons(description=' ', options=('Solve', 'Change Parameters'), value='Solve'), I…

<function __main__.set_weight(solve: bool, p1: int, c: int)>

### Presentation of sheduling obtained after hiring extra-nurses

In [11]:
def interactive_bis(nurse : str, day : str, shift : str):
    
    
    if nurse == 'All':
        df_int = solution_bis[solution_bis['Nurse'].isin(I_2)]
    else:
        df_int = solution_bis[solution_bis['Nurse']==nurse]

    if day == 'All':
        df_int = df_int[df_int['Day'].isin(T)]
    else:
        df_int = df_int[df_int['Day']==day]

    if shift == 'All':
        df_int = df_int[df_int['Shift'].isin(S)]
    else:
        df_int = df_int[df_int['Shift']==shift]


    df_int = df_int.set_index(pd.Index(list(range(1, len(df_int.index)+1))))
    display(df_int)
    

def display_scheduling(display):
    if display == True:
        interact(interactive_bis, nurse = widgets.Dropdown(value = 'All', options = list(I_2) + ['All']), 
                 day = widgets.Dropdown(value = 'All', options = list(T)+ ['All']), 
                 shift = widgets.Dropdown(value = 'All', options = list(S)+ ['All'])
                );
        
        
interact(display_scheduling, display = widgets.ToggleButton(value=False, description = 'Display'))

interactive(children=(ToggleButton(value=False, description='Display'), Output()), _dom_classes=('widget-inter…

<function __main__.display_scheduling(display)>